In [19]:
# Read 'chats' from an input file.
chats = []
filepath = "/Path/To/Exported/Chat/File.txt"
with open(filepath, 'r') as f:
    for line in f:
        chats.append(line.strip('\n '))

In [20]:
def is_valid(line):
    if ': ' in line and ' - ' in line:
        return True
    return False

# Handle multiline chats.
modified_chats = []
for chat in chats:
    if is_valid(chat):
        modified_chats.append(chat)
    else:
        # If line is not a new chat, add it to an earlier chat line. 
        modified_chats[-1] += " "
        modified_chats[-1] += chat
        
chats = modified_chats

In [21]:
# Convert chat strings into tuple (<date>,<time>,<speaker>,<speech>)
org_chats = []

for chat in chats:
    date_time = chat.split(' - ')[0]
    speaker_speech = chat.split(' - ')[1]
    date = date_time.split(', ')[0]
    time = date_time.split(', ')[1]
    speaker_speech = speaker_speech.split(': ')
    if len(speaker_speech) < 2:
        continue
    speaker = speaker_speech[0]
    speech = speaker_speech[1]
    org_chats.append((date, time, speaker, speech))

chats = org_chats

In [22]:
# Get messages by day and speaker.
def get_message_stats_by_day_speaker(chats):
    message_stats = {}
    for chat_tuple in chats:
        (date, time, speaker, speech) = chat_tuple
        if date not in message_stats:
            message_stats[date] = {}
            message_stats[date][speaker] = 1
        else:
            if speaker not in message_stats[date]:
                message_stats[date][speaker] = 1
            else:
                message_stats[date][speaker] += 1
    return message_stats

message_stats = get_message_stats_by_day_speaker(chats)

In [23]:
# Plot message count and distributions as stacked bar.
def datesort(date):
    dd, mm, yy = date.split('/')
    return yy+mm+dd

x = []
y1 = []
y2 = []
names = []
for key in sorted(message_stats.keys(), key=datesort):
    x.append(key)
    count = 0
    val = message_stats[key]
    names = []
    for key in val.keys():
        names.append(key)
    names = sorted(names)
    if len(names) < 2:
        continue
    y1.append(val[names[0]])
    y2.append(val[names[1]])

import plotly.graph_objects as go
fig = go.Figure(data=[go.Bar(name=names[0], y=y1, x=x), go.Bar(name=names[1], y=y2, x=x)])
fig.update_layout(barmode='stack', title_text='Message count and distribution')
fig.write_html('message_counts_distribution.html', auto_open=True)